In [2]:
from fractions import Fraction
import heapq

# ハフマン符号

## Node クラス

出現した文字の関係性を表すノード

### メソッド

- character: str<br/>
  出現した文字(または子に含まれる文字(識別用))

- probability: float<br/>
  出現した文字(または子に含まれる文字(識別用))の出現回数

- left: Node | None<br/>
  左の子

- right: Node | None<br/>
  右の子


In [3]:
class Node:
    def __init__(self, character: str, probability: float, left, right) -> None:
        self.character = character
        self.probability = probability
        self.left: Node | None = left
        self.right: Node | None = right

    def __lt__(self, other):
        return self.probability < other.probability

## 関数

### to_bit 関数

いい感じにハフマン符号化してくれる関数


In [4]:
def to_bit(node: Node, bit: str, bits: dict[str, str]):
    if node.left is None and node.right is None:
        bits[node.character] = bit
    if node.left is not None:
        to_bit(node.left, bit + "1", bits)
    if node.right is not None:
        to_bit(node.right, bit + "0", bits)

### encode 関数

1. 文字列`string`を受け取る
2. 含まれる文字をカウントする
3. ハフマン符号用の木にする
4. 木を解析して、それぞれの文字を符号にする
5. 文字列(`string`)をエンコードして返す


In [5]:
def encode(string: str):
    dictionary: dict[str, int] = {}
    total_chars = len(string)
    for s in string:
        if s in dictionary:
            dictionary[s] += 1
        else:
            dictionary[s] = 1
    print("各文字の出現頻度:", dictionary)
    print("各文字の出現確率:")
    for char, freq in dictionary.items():
        print(f"'{char}': {Fraction(freq, total_chars)}")
    nodes: list[Node] = []
    heapq.heapify(nodes)
    for s in dictionary:
        heapq.heappush(nodes, Node(s, dictionary[s], None, None))
    while len(nodes) > 1:
        left = heapq.heappop(nodes)
        right = heapq.heappop(nodes)
        new_node = Node(
            left.character + right.character,
            left.probability + right.probability,
            left,
            right,
        )
        heapq.heappush(nodes, new_node)
    top = heapq.heappop(nodes)
    bits: dict[str, str] = {}
    to_bit(top, "", bits)
    print("文字のビット表現:", bits)
    result: str = ""
    for s in string:
        result += bits[s]
    print("符号化されたビット列の長さ:", len(result))
    return result


# 実行例
print(encode("GGCCGGGCGCGGTGGCTCACGCCTGTAATC"))

各文字の出現頻度: {'G': 12, 'C': 10, 'T': 5, 'A': 3}
各文字の出現確率:
'G': 2/5
'C': 1/3
'T': 1/6
'A': 1/10
文字のビット表現: {'G': '1', 'A': '011', 'T': '010', 'C': '00'}
符号化されたビット列の長さ: 56
11000011100100110101100010000110010000010101001101101000
